In [1]:
# paramters for H2 at R=1.75A
g0 = -0.5597
g1 = +0.1615
g2 = -0.0166
g3 = +0.4148
g4 = +0.1226
g5 = +0.1226
nuclear_repulsion = 0
# nuclear_repulsion = 0.3023869942
Energy_FCI = -0.97516853

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, transpile

from qiskit_aer import QasmSimulator
# from qiskit_aer.noise import NoiseModel, pauli_error

# from qiskit.providers.aer.noise import NoiseModel, depolarizing_error, pauli_error
# from qiskit import Aer, execute

from scipy.optimize import OptimizeResult

In [4]:
import math
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from scipy.optimize import OptimizeResult


def get_probability_distribution(counts, shots):
    # Probability Distribution
    for k in {'00', '01', '10', '11'}:
        if k not in counts.keys():
            counts[k] = 0 # if k isn't already a key in 'counts', add k to counts with a value of 0
    sorted_counts = sorted(counts.items()) # sort the items (key-value pairs) by key
    # print("Sorted Counts:", counts)

    output_distr = [ v[1] / shots for v in sorted_counts ] # normalize
    if (len(output_distr) == 1):
        output_distr.append(1-output_distr[0]) # handle single outcome case
    # print("Output Distribution", output_distr)
    
    return output_distr


def func_and_gradient(x_opt, fun, eps):
    f = fun(x_opt)
    grad = np.zeros_like(x_opt)

    for i in range(x_opt.size):
        x_plus_h = x_opt.copy()
        x_plus_h[i] += eps
        f_plus_h = fun(x_plus_h)
        # print(f_plus_h, f)
        grad[i] = (f_plus_h - f) / eps
    print('eps', eps)
    return f, grad


def adam(fun, x0, jac=None, iters=100, options=None, beta1=0.9, beta2=0.999, epsilon=1e-8, eps=0.02):
    # learning rate
    base_lr = 0.1

    # Initialize the ADAM variables
    t = 0
    m_t = 0
    v_t = 0

    # Initialize the optimization result
    nit = 0
    nfev = 0
    njev = 0
    success = False
    message = ''
    fun_val = np.inf
    x_opt = np.copy(x0)

    if options is None:
        options = {}
    maxiter = options.get('maxiter', iters)

    while nit < maxiter:
        # Compute the function value and gradient
        fval, grad = func_and_gradient(x_opt, fun, eps)  # approx_fprime(x_opt, fun, eps)
        nfev += 2
        njev += 1

        fun_val = fval
        # Compute the ADAM update
        t += 1
        m_t = beta1 * m_t + (1 - beta1) * grad
        v_t = beta2 * v_t + (1 - beta2) * grad ** 2
        m_t_hat = m_t / (1 - beta1 ** t)
        v_t_hat = v_t / (1 - beta2 ** t)

        lr_current = 0.5 * base_lr * (math.cos(math.pi * nit / maxiter) + 1)
        lr = lr_current / (np.sqrt(v_t_hat) + epsilon)

        # Update the parameters
        x_opt = x_opt - lr * m_t_hat

        nit += 1

    result = OptimizeResult(fun=fun_val, x=x_opt, nit=nit, nfev=nfev, njev=njev, success=success, message=message)

    return result

class history:
    def __init__(self):
        self.thetas = []
        self.energies = []
        self.P1 = []
        self.P2 = []
        self.P3 = []


def VQE_H2(ansatz, backend, theta, shots, history):
    # paramters for H2 at R=1.75A
    g0 = -0.5597
    g1 = +0.1615
    g2 = -0.0166
    g3 = +0.4148
    g4 = +0.1226
    g5 = +0.1226
    nuclear_repulsion = 0
    # nuclear_repulsion = 0.3023869942
    Energy_FCI = -0.97516853
    
    E = g0 + nuclear_repulsion
    shots1, shots2, shots3 = shots
    
    ansatz_1 = ansatz.copy()
    ansatz_1.measure([0,1], [0,1])
    ansatz_1_t = transpile(ansatz_1, backend)

    counts = backend.run(ansatz_1_t, shots=shots1).result().get_counts(ansatz_1_t)
    output_distr = get_probability_distribution(counts, shots1)
    # print(output_distr)
    E1 = -g1 * (output_distr[0] + output_distr[1] - output_distr[2] - output_distr[3])
    E2 = -g2 * (output_distr[0] - output_distr[1] + output_distr[2] - output_distr[3])
    E3 = g3 * (output_distr[0] - output_distr[1] - output_distr[2] + output_distr[3])
    E += E1 + E2 + E3
    history.P1.append(output_distr)



    ansatz_2 = ansatz.copy()
    ansatz_2.sdg(1)
    ansatz_2.h(1)
    ansatz_2.sdg(0)
    ansatz_2.h(0)
    ansatz_2.measure([0,1], [0,1])
    ansatz_2_t = transpile(ansatz_2, backend)

    counts = backend.run(ansatz_2_t, shots=shots1).result().get_counts(ansatz_2_t)
    output_distr = get_probability_distribution(counts, shots2)
    print(output_distr)
    E += g4 * (output_distr[0] - output_distr[1] - output_distr[2] + output_distr[3])
    history.P2.append(output_distr)


    ansatz_3 = ansatz.copy()
    ansatz_3.h(1)
    ansatz_3.h(0)
    ansatz_3.measure([0,1], [0,1])
    ansatz_3_t = transpile(ansatz_3, backend)
    counts = backend.run(ansatz_3_t, shots=shots3).result().get_counts(ansatz_3_t)
    output_distr = get_probability_distribution(counts, shots3)
    print(output_distr)
    E += g5 * (output_distr[0] - output_distr[1] - output_distr[2] + output_distr[3])
    history.P3.append(output_distr)

    history.thetas.append(theta)
    history.energies.append(E)
    
    print('theta track: ', history.thetas[-1])
    print('energy track: ', history.energies[-1])
    print("Len: ", len(history.energies))

    return E

# VQE_H2(ansatz)

In [ ]:
with Session(backend=backend) as session:
    

In [5]:
ansatz = QuantumCircuit(2,2)
ansatz.name = 'H2 STO-3G g1-g3'
ansatz.x(0)
ansatz.ry(np.pi/2,1)
ansatz.rx(3*np.pi/2,0)
ansatz.cx(1,0)
ansatz.rz(theta[0],0)
ansatz.cx(1,0)
ansatz.ry(3*np.pi/2,1)
ansatz.rx(np.pi/2,0)

backend = AerSimulator()

theta_test = [-2]
record_history = history()
shots = 600
per_shots = shots//3

result = adam(lambda x: VQE_H2(ansatz, backend, x, shots=(per_shots,per_shots,per_shots), history=record_history), theta_test, iters=500, eps=0.02)


NameError: name 'theta' is not defined